# Bootstrap Sets: Class Centroids and Boundary Lines
calculate class centroids for the training set of each bootstrap data set, store them in a dataframe. Output each centroid output into a storage directory. Copy boundary line output in bootstrap set directories to the same storage directory by giving them a name with set number. Hence the storage directory will have the centroids along with the boundary line coefficients for each set.  

In [2]:
using Glob, CSV, DataFrames, Statistics

In [ ]:
searchdir(path,key) = filter(x->occursin(key,x), readdir(path))

searchdir (generic function with 1 method)

In the output directory holding a full analysis results (single, loocv, kfoldcv, bs etc directories). It will create a directory named "boundaries_centroids" in the folder, which will be the address to write the centroid and boundary line outputs.

Also modify the script below if the classes are different than 0 and 2, which is the assumption below.

In [3]:
cd("C:/dino_nature/output/2023-11-07-D1/bs")
mkdir("boundaries_centroids")

for i in 1:100
    inpfile = joinpath(".", "set_$i", "femur_set_$i.csv")
    data = DataFrame(CSV.File(inpfile))
    data = filter(:"diving.or.not" =>  !=("UNKNOWN"), data)   # eliminate the test taxa, thus keep only the training set
    data_0 = filter(:"diving.or.not" =>  ==("0"), data)[:, [:"Global.compactness", :"MD.log"]]    # extract a class
    data_2 = filter(:"diving.or.not" =>  ==("2"), data)[:, [:"Global.compactness", :"MD.log"]]    # extract a class
    mean_0 = mean.(eachcol(data_0))     # calculate the mean of each column for the given class 
    mean_2 = mean.(eachcol(data_2))     # calculate the mean of each column for the given class
    centroids = transpose(hcat(mean_0, mean_2))   #  combine the centroids of each class in a matrix for converting to a dataframe
    centroids = DataFrame(centroids, :auto)       #  create a dataframe of centroids
    rename!(centroids, [:"Cg", :"logD"])
    outfile = joinpath(".", "boundaries_centroids", "centroids_$i.csv")
    CSV.write(outfile, centroids)
    # now copy centroids to the folder by renaming them with the set index:
    cp(joinpath(".", "set_$i", "BOUNDARY_LINE_COEFFS.csv"), joinpath(".", "boundaries_centroids", "BOUNDARY_LINE_COEFFS_$i.csv"), force=true)
    #println(outfile)
end